In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests 

import warnings
warnings.filterwarnings('ignore')

#### 1. Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

In [2]:
auc_data = pd.read_csv('AUCs.txt', header=0, delimiter='\t')
auc_data.rename(columns={'Unnamed: 0': 'Dataset'}, inplace=True)

aucs = auc_data.drop(auc_data.columns[0], axis=1)

auc_data.head()

,Dataset,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [3]:
%%time 

stats_data = []

for i, i_column in enumerate(aucs.columns):
    for j, j_column in enumerate(aucs.columns):
        if i >= j:
            continue
        w_stat, p = stats.wilcoxon(aucs.iloc[:, i], aucs.iloc[:, j])
        
        stats_data.append([i_column, j_column, w_stat, p])
        
auc_correlation = pd.DataFrame.from_records(stats_data)
auc_correlation.columns = ['Model i', 'Model j', 'Wilcoxon', 'p-value']
auc_correlation

CPU times: user 16.4 ms, sys: 170 µs, total: 16.6 ms
Wall time: 14.6 ms


,Model i,Model j,Wilcoxon,p-value
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


#### 2. Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [4]:
reject, p_corrected, a1, a2 = multipletests(auc_correlation['p-value'], 
                                            alpha = 0.05, 
                                            method = 'holm') 

auc_correlation['p_corrected'] = p_corrected
auc_correlation['reject'] = reject

auc_correlation

,Model i,Model j,Wilcoxon,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


#### 3. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

In [5]:
reject, p_corrected, a1, a2 = multipletests(auc_correlation['p-value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

auc_correlation['p_corrected'] = p_corrected
auc_correlation['reject'] = reject

auc_correlation

,Model i,Model j,Wilcoxon,p-value,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True
